In [ ]:
!pip install openai --q
!pip install langchain --q
!pip install unstructured --q
!pip install unstructured libmagic python-magic python-magic-bin --q
!pip install faiss -cpu
!pip install sentance-transformers
!pip install gradio
!pip install streamlit
!pip install faiss-gpu
!pip install pickle
!pip install tiktoken

In [2]:
import os
import gradio as gr
import pickle
import time
from langchain import OpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS



/usr/local/lib/python3.10/dist-packages/langchain/__init__.py:24: UserWarning: Importing OpenAI from langchain root module is no longer supported.
  warnings.warn(


In [3]:
os.environ['OPENAI_API_KEY']='****'

In [28]:
llm=OpenAI(temperature=0.7,max_tokens=500)
loader=UnstructuredURLLoader(urls=[
    "https://economictimes.indiatimes.com/industry/banking/finance/crypto-drama-unfolds-binance-bnb-connects-goodbye-bitcoin-btcs-silent-play-and-pomerdoge-pomds-rising-star/articleshow/103142105.cms",
    "https://en.m.wikipedia.org/wiki/Jubilant_FoodWorks#:~:text=Jubilant%20FoodWorks%20Limited%20is%20an,for%20Dunkin'%20Donuts%20in%20India"
])
data=loader.load()

In [29]:
#Splitting into chunks
r_splitter=RecursiveCharacterTextSplitter(
    separators=["\n\n","\n","."," "], #diff between recursive text split and text split is that we can include a variety of separators and not just one.
    chunk_size=1000,
    chunk_overlap=100
)
"""
First text sep w \n\n and then if size still more than allowed chunk size then split again with \n and then . and so on
Merge also happens in this step where the chunks that are too small it gets merged so they overlap a little.
"""
docs=r_splitter.split_documents(data)
len(docs)

44

In [30]:
#Embeddings
embeddings=OpenAIEmbeddings()
vectorstore_openai=FAISS.from_documents(docs,embeddings)

In [31]:
#Save FAISS index to pickle file
#Store vector in local
file_path="faiss_store_openai.pkl"
with open(file_path,"wb") as f:
  pickle.dump(vectorstore_openai,f)


In [45]:
def querys(query):
  if query:
    if os.path.isfile(file_path):
      with open(file_path,"rb") as f:
        vectorstore=pickle.load(f)
        chain=RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorstore.as_retriever())
        result=chain({"question":query},return_only_outputs=True)
        #print(result['answer'])#op
        return result['answer'],result['sources']

In [48]:
iface=gr.Interface(
    fn=querys,
    inputs=gr.Textbox(label="Enter query below"),
    outputs=[gr.Textbox(label="Answer"),gr.Textbox(label="Source")],
    title="Article Research Tool"

)

In [49]:
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://7c7b4e640cf8b9feff.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
